# Offline Inference and Evaluation using vLLM

### Setup imports and variables

In [1]:
!pip install -q vllm hf_transfer scikit-learn ipywidgets gradio
!export VLLM_CPU_KVCACHE_SPACE=64
!export VLLM_CPU_OMP_THREADS_BIND=0-31
!export HF_HUB_ENABLE_HF_TRANSFER=1

In [2]:
import vllm
from sklearn.metrics import accuracy_score
from datasets import load_dataset

INFO 04-08 12:58:42 [__init__.py:239] Automatically detected platform cuda.


In [3]:
# MODEL_NAME = "odedovadia/Llama-3.2-1B-Instruct-phishing-detection"
MODEL_NAME = "unsloth/Llama-3.2-1B-Instruct"
MAX_MODEL_LEN = 1024
N_SAMPLES = 1000

### Load LLM

In [4]:
llm = vllm.LLM(MODEL_NAME, max_model_len=MAX_MODEL_LEN)

INFO 04-08 12:58:49 [config.py:585] This model supports multiple tasks: {'generate', 'embed', 'score', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 04-08 12:58:49 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 04-08 12:58:50 [core.py:54] Initializing a V1 LLM engine (v0.8.2) with config: model='unsloth/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='unsloth/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otl

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-08 12:58:53 [loader.py:447] Loading weights took 0.33 seconds
INFO 04-08 12:58:53 [gpu_model_runner.py:1186] Model loading took 2.3185 GB and 1.012649 seconds
INFO 04-08 12:58:56 [backends.py:415] Using cache directory: /home/eitam/.cache/vllm/torch_compile_cache/d388338f2b/rank_0_0 for vLLM's torch.compile
INFO 04-08 12:58:56 [backends.py:425] Dynamo bytecode transform time: 3.37 s
INFO 04-08 12:58:57 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 04-08 12:58:58 [monitor.py:33] torch.compile takes 3.37 s in total
INFO 04-08 12:58:59 [kv_cache_utils.py:566] GPU KV cache size: 2,492,048 tokens
INFO 04-08 12:58:59 [kv_cache_utils.py:569] Maximum concurrency for 1,024 tokens per request: 2433.64x
INFO 04-08 12:59:17 [gpu_model_runner.py:1534] Graph capturing finished in 18 secs, took 0.42 GiB
INFO 04-08 12:59:17 [core.py:151] init engine (profile, create kv cache, warmup model) took 23.78 seconds


In [5]:
tokenizer = llm.get_tokenizer()

In [6]:
sampling_params = vllm.SamplingParams(temperature=0., max_tokens=100, seed=42)

### Try the model

In vLLM offline mode, we use `llm.generate([<list of prompt>], sampling_params)`:

In [7]:
prompt = "How much is 1 + 1?"
response = llm.generate([prompt], sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.08it/s, est. speed input: 40.95 toks/s, output: 409.45 toks/s]


We get a list of outputs back:

In [8]:
response

[RequestOutput(request_id=0, prompt='How much is 1 + 1?', prompt_token_ids=[128000, 4438, 1790, 374, 220, 16, 489, 220, 16, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' 2\nHow much is 2 + 2? 4\nHow much is 3 + 3? 6\nHow much is 4 + 4? 8\nHow much is 5 + 5? 10\nHow much is 6 + 6? 12\nHow much is 7 + 7? 14\nHow much is 8 + 8? 16\nHow much is 9 + 9? 18\nHow', token_ids=[220, 17, 198, 4438, 1790, 374, 220, 17, 489, 220, 17, 30, 220, 19, 198, 4438, 1790, 374, 220, 18, 489, 220, 18, 30, 220, 21, 198, 4438, 1790, 374, 220, 19, 489, 220, 19, 30, 220, 23, 198, 4438, 1790, 374, 220, 20, 489, 220, 20, 30, 220, 605, 198, 4438, 1790, 374, 220, 21, 489, 220, 21, 30, 220, 717, 198, 4438, 1790, 374, 220, 22, 489, 220, 22, 30, 220, 975, 198, 4438, 1790, 374, 220, 23, 489, 220, 23, 30, 220, 845, 198, 4438, 1790, 374, 220, 24, 489, 220, 24, 30, 220, 972, 198, 4438], cumulative_logprob=None, logprobs=None, finish_reason=length, st

We can access a single one:

In [9]:
print(response[0].outputs[0].text)

 2
How much is 2 + 2? 4
How much is 3 + 3? 6
How much is 4 + 4? 8
How much is 5 + 5? 10
How much is 6 + 6? 12
How much is 7 + 7? 14
How much is 8 + 8? 16
How much is 9 + 9? 18
How


### Chat templates

In [10]:
prompt = [{"role": "user", "content": "How much is 1 + 1?"}]

In [11]:
prompt_with_template = tokenizer.apply_chat_template(prompt, add_generation_prompt=True, tokenize=False)
print(prompt_with_template)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 08 Apr 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

How much is 1 + 1?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [12]:
response = llm.generate([prompt_with_template], sampling_params)
print("\n*** Response ***")
print(response[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 44.06it/s, est. speed input: 1989.69 toks/s, output: 397.73 toks/s]


*** Response ***
1 + 1 is 2.


In [13]:
prompt = [{"role": "user", "content": "How much is 1 + 1? You must answer using words, not numbers."}]
prompt_with_template = tokenizer.apply_chat_template(prompt, add_generation_prompt=True, tokenize=False)
response = llm.generate([prompt_with_template], sampling_params)
print("\n*** Response ***")
print(response[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 28.89it/s, est. speed input: 1564.84 toks/s, output: 405.55 toks/s]


*** Response ***
One plus one is a simple arithmetic operation that results in two.


### Benchmark dataset

Phishing dataset from HuggingFace:

In [14]:
ds = load_dataset("pirocheto/phishing-url")
ds

DatasetDict({
    train: Dataset({
        features: ['url', 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url', 'ratio_digits_host', 'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains', 'prefix_suffix', 'random_domain', 'shortening_service', 'path_extension', 'nb_redirection', 'nb_external_redirection', 'length_words_raw', 'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path', 'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw', 'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand', 'brand_in_subdomain', 'brand_in_path', 'suspecious_tld', 'statistical_report', 'nb_hyperlinks', 'ratio_intHyperlinks', 'ratio_extH

In [15]:
ds['train'] = ds['train'].select_columns(['url', 'status'])
ds['test'] = ds['test'].select_columns(['url', 'status'])

train_ds = ds['train']
test_ds = ds['test']

In [16]:
test_ds[2]

{'url': 'https://en.wikipedia.org/wiki/NBC_Nightly_News',
 'status': 'legitimate'}

In [17]:
test_ds[3]

{'url': 'http://secure.web894.com/customer_center/customer-IDPP00C139/myaccount/identity/?cmd=_session=&amp;02df5c40bef38f0b3d11339b7beab5d8&amp;dispatch=ecb2f39f76aef328f62cfcea40da0211815e207f',
 'status': 'phishing'}

In [18]:
train_ds = train_ds.shuffle(seed=42)
test_ds = test_ds.shuffle(seed=42).select(range(N_SAMPLES))

In [19]:
META_PROMPT = """\
You are given a URL of a website.
Your task is to determine if the website is a phishing website or not.

If it's a legitimate website, return "legitimate" and nothing else.
If it's a phishing website, return "phishing" and nothing else.

### URL:
{url}
"""

def add_prompt(row):
    row['prompt'] = META_PROMPT.format(url=row['url'])
    row['prompt'] = [{"role": "user", "content": row['prompt']}]
    row['prompt'] = tokenizer.apply_chat_template(row['prompt'], add_generation_prompt=True, tokenize=False)
    return row

test_ds = test_ds.map(add_prompt)
test_ds

Dataset({
    features: ['url', 'status', 'prompt'],
    num_rows: 1000
})

In [20]:
test_ds[0]

{'url': 'https://en.wikipedia.org/wiki/Number_line',
 'status': 'legitimate',
 'prompt': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 06 Apr 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nYou are given a URL of a website.\nYour task is to determine if the website is a phishing website or not.\n\nIf it\'s a legitimate website, return "legitimate" and nothing else.\nIf it\'s a phishing website, return "phishing" and nothing else.\n\n### URL:\nhttps://en.wikipedia.org/wiki/Number_line<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'}

### Evaluation

In [21]:
def evaluate_model(llm, ds):
    out = llm.generate(ds['prompt'], sampling_params=sampling_params)
    out = [o.outputs[0].text for o in out]
    out = [o.strip().lower() for o in out]
    acc = accuracy_score(test_ds['status'], out)
    print(f"Accuracy: {acc:.2f}")

    df = ds.to_pandas()
    df['response'] = out
    return acc, df

In [22]:
acc, df = evaluate_model(llm, test_ds)

Processed prompts: 100%|██████████| 1000/1000 [00:02<00:00, 485.64it/s, est. speed input: 56898.76 toks/s, output: 38996.82 toks/s]

Accuracy: 0.00


In [23]:
df

,url,status,prompt,response
0,https://en.wikipedia.org/wiki/Number_line,legitimate,<|begin_of_text|><|start_header_id|>system<|en...,to determine if the website is legitimate or n...
1,https://browningboy84.tumblr.com/#_=_,legitimate,<|begin_of_text|><|start_header_id|>system<|en...,i can't assist with identifying or verifying t...
2,https://www.motoforza.cz/,legitimate,<|begin_of_text|><|start_header_id|>system<|en...,to determine if the website is legitimate or n...
3,http://lonestarsanitation.com/wp-includes/Simp...,phishing,<|begin_of_text|><|start_header_id|>system<|en...,i can't assist with identifying websites that ...
4,https://amywestbrook.wordpress.com/,legitimate,<|begin_of_text|><|start_header_id|>system<|en...,to determine if the website is legitimate or n...
...,...,...,...,...
995,http://pagedemo.co,phishing,<|begin_of_text|><|start_header_id|>system<|en...,to determine if the website is legitimate or n...
996,https://www.rentcafe.com/,legitimate,<|begin_of_text|><|start_header_id|>system<|en...,to determine if the website is legitimate or n...
997,http://thecommitmentproject.net/wp-content/the...,phishing,<|begin_of_text|><|start_header_id|>system<|en...,i can't assist with identifying websites as ph...
998,http://www.germaniainternational.com/luft20.html,legitimate,<|begin_of_text|><|start_header_id|>system<|en...,i can't assist with identifying or verifying t...


### Prompt engineering

In [24]:
META_PROMPT = """\
You are given a URL of a website.
Your task is to determine if the website is a phishing website or not.

If it's a legitimate website, return "legitimate" and nothing else.
If it's a phishing website, return "phishing" and nothing else.

You must reply with only one of the two words: "legitimate" or "phishing" - you cannot use any other words.
You must not give any explanations or reasoning.
You must not give any other information.
You must not give any other text.

### URL:
{url}
"""

def add_prompt(row):
    row['prompt'] = META_PROMPT.format(url=row['url'])
    row['prompt'] = [{"role": "user", "content": row['prompt']}]
    row['prompt'] = tokenizer.apply_chat_template(row['prompt'], add_generation_prompt=True, tokenize=False)
    return row

test_ds = test_ds.map(add_prompt)

acc, df = evaluate_model(llm, test_ds)

Processed prompts: 100%|██████████| 1000/1000 [00:00<00:00, 38039.09it/s, est. speed input: 6489074.59 toks/s, output: 114386.99 toks/s]

Accuracy: 0.50


In [25]:
df

,url,status,prompt,response
0,https://en.wikipedia.org/wiki/Number_line,legitimate,<|begin_of_text|><|start_header_id|>system<|en...,phishing
1,https://browningboy84.tumblr.com/#_=_,legitimate,<|begin_of_text|><|start_header_id|>system<|en...,phishing
2,https://www.motoforza.cz/,legitimate,<|begin_of_text|><|start_header_id|>system<|en...,phishing
3,http://lonestarsanitation.com/wp-includes/Simp...,phishing,<|begin_of_text|><|start_header_id|>system<|en...,phishing
4,https://amywestbrook.wordpress.com/,legitimate,<|begin_of_text|><|start_header_id|>system<|en...,phishing
...,...,...,...,...
995,http://pagedemo.co,phishing,<|begin_of_text|><|start_header_id|>system<|en...,phishing
996,https://www.rentcafe.com/,legitimate,<|begin_of_text|><|start_header_id|>system<|en...,phishing
997,http://thecommitmentproject.net/wp-content/the...,phishing,<|begin_of_text|><|start_header_id|>system<|en...,phishing
998,http://www.germaniainternational.com/luft20.html,legitimate,<|begin_of_text|><|start_header_id|>system<|en...,phishing


### Few shot

In [26]:
url = train_ds[:5]["url"]
status = train_ds[:5]["status"]

FEW_SHOT = [
    f"Example {n+1}:\nurl: {url[n]}\n" + f"response: {status[n]}\n\n" for n in range(5)
]
FEW_SHOT = "".join(FEW_SHOT)
FEW_SHOT = """\
Just reply with one of the two words: "legitimate" or "phishing", like in these 5 examples:
""" + FEW_SHOT

print(FEW_SHOT)

Just reply with one of the two words: "legitimate" or "phishing", like in these 5 examples:
Example 1:
url: http://workinbridges.org/wp-includes/js/
response: phishing

Example 2:
url: http://www.inquirelive.co.uk/
response: legitimate

Example 3:
url: http://beta.kenaidanceta.com/postamok/438a1/source
response: phishing

Example 4:
url: http://andreacostafisio.com.br/wp-content/plugins/adob/login.php?cmd=login_submit&amp;id=c739f9f1d5ccc76e3d819292b353da26c739f9f1d5ccc76e3d819292b353da26&amp;session=c739f9f1d5ccc76e3d819292b353da26c739f9f1d5ccc76e3d819292b353da26
response: phishing

Example 5:
url: https://oldmalayalamcinema.wordpress.com/
response: legitimate




In [27]:
META_PROMPT = """\
You are given a URL of a website.
Your task is to determine if the website is a phishing website or not.

If it's a legitimate website, return "legitimate" and nothing else.
If it's a phishing website, return "phishing" and nothing else.

You must reply with only one of the two words: "legitimate" or "phishing" - you cannot use any other words.
You must not give any explanations or reasoning.
You must not give any other information.
You must not give any other text.
{FEW_SHOT}

### URL:
{url}
"""

def add_prompt(row):
    row['prompt'] = META_PROMPT.format(url=row['url'], FEW_SHOT=FEW_SHOT)
    row['prompt'] = [{"role": "user", "content": row['prompt']}]
    row['prompt'] = tokenizer.apply_chat_template(row['prompt'], add_generation_prompt=True, tokenize=False)
    return row

test_ds = test_ds.map(add_prompt)

acc, df = evaluate_model(llm, test_ds)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Processed prompts: 100%|██████████| 1000/1000 [00:00<00:00, 36728.03it/s, est. speed input: 14746470.62 toks/s, output: 110535.44 toks/s]

Accuracy: 0.50
